Let's see if we can add our own CMAP to gmx, using ParmEd
Steps:
- load gmx topology with parmed
- identify the indices (and possibly the atom objects) of the atoms that form the backbone dihedrals
- we need to match the mdtraj atoms to the parmed ones
- add cmaptypes and cmaps to parmed topology and export 


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from copy import copy
import numpy as np
import parmed as pmd
import mdtraj as mdt
from parmed.topologyobjects import CmapType, Cmap

In [4]:
chm_pmd_top = pmd.load_file("charmm_topol.top", xyz="step3_input.gro")

In [5]:
for cmap_type in chm_pmd_top.cmap_types:
    print(np.array(cmap_type.grid._data).min())

-7.05
-6.205900095602294
-1.2565999043977056
-6.205900095602294
-7.05


In [6]:
import sys
sys.path.append("../../")
from data_utils import build_cmap_atoms

/home/gzappavigna/miniforge3/envs/lab/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TODO: match mdtraj atoms to parmed ones 

1) parmed atom -> mdtraj atom
2) mdtraj atom -> mdtraj index (for the angles array)


In [7]:

basepath = "/data/gzappavigna/lmp_conf_builder/gmx/amber/"

# ref_traj = mdt.iterload(basepath + "step5_production_nojump.xtc", top=basepath + "step5_production_nojump.pdb")
ref_traj = mdt.load(basepath + "step5_production_nojump.xtc", top=basepath + "step5_production_nojump.pdb")


In [8]:
basepath = "/data/gzappavigna/lmp_conf_builder/runs/"

ens_traj = mdt.load(basepath + "ensemble.xtc", top=basepath + "cg.pdb")

/home/gzappavigna/miniforge3/envs/lab/lib/python3.12/site-packages/mdtraj/formats/pdb/pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


In [9]:
from cmap import build_phipsi

ens_phipsi = build_phipsi(ens_traj)
ref_phipsi = build_phipsi(ref_traj)


In [24]:
pmd_top = pmd.load_file("amber_topol.top", xyz="6921a_solv_ions.gro", defines={"POSRES": 1})

In [26]:
pmd_top.itps

['amber99sb.ff/forcefield.itp',
 'posre.itp',
 'amber99sb.ff/tip4p.itp',
 'amber99sb.ff/ions.itp']

In [27]:
cmap_atoms_list = build_cmap_atoms([[getattr(dih, f"atom{i+1}") for i in range(4)] for dih in pmd_top.dihedrals])

In [28]:
id_to_mdt_atom = {(atom.residue.index, atom.name): atom for atom in ref_traj.top.atoms}

residueNameReplacements = mdt.formats.pdb.PDBTrajectoryFile._residueNameReplacements
atomNameReplacements = mdt.formats.pdb.PDBTrajectoryFile._atomNameReplacements

pmd_atoms = [atom for atom in pmd_top.atoms if atom.residue.name not in {"SOL", "CL", "NA"}]

pmd_to_mdt_atom = {}

for pmd_atom in pmd_atoms:
    res = pmd_atom.residue
    mdt_resname = residueNameReplacements[res.name]
    mdt_name = atomNameReplacements[mdt_resname].get(pmd_atom.name, pmd_atom.name)
    mdt_atom = id_to_mdt_atom[(res.idx, mdt_name)]

    pmd_to_mdt_atom[pmd_atom] = mdt_atom

In [29]:
from cmap import build_phipsi, build_cmap

# ens_phipsi = build_phipsi(ens_traj)
# ref_phipsi = build_phipsi(ref_traj)

cmap_types = []
cmap_objs = []

for i, cmap_atoms in enumerate(cmap_atoms_list):
    mdt_atoms = [pmd_to_mdt_atom[atom] for atom in cmap_atoms]
    # the third is the CA
    cmap_ind = mdt_atoms[2].residue.index

    for atom in cmap_atoms[1:-1]:
        atom_type = copy(atom.atom_type)
        new_name = atom_type.name + f"_{i+1}"
        atom_type.name = new_name
        atom.atom_type = atom_type
        atom.type = new_name

    assert mdt_atoms[2].name == "CA"
    resname = mdt_atoms[2].residue.name
    print(cmap_ind)

    sel_ens_phipsi = ens_phipsi[cmap_ind]
    sel_ref_phipsi = ref_phipsi[cmap_ind]

    # print(sel_ens_phipsi.max() - np.pi, sel_ens_phipsi.min() + np.pi)

    cmap, *_ = build_cmap(sel_ens_phipsi, sel_ref_phipsi, ens_bw=0.4, ref_bw=0.2)
    cmap_type = CmapType(resolution=24, grid=cmap.flatten().tolist(), comments=[resname])
    cmap_type.used = True
    cmap_obj = Cmap(*cmap_atoms, type=cmap_type)
    cmap_types.append(cmap_type)
    cmap_objs.append(cmap_obj)


1
0.4296686589853693
(24, 24)
0.46056986455430776
(24, 24)
-20.573479312325365
-20.573479301481328
-10.45764016810443
2
0.28501525756878204
(24, 24)
0.4215412531523982
(24, 24)
-20.573479271100037
-20.573479128146946
-10.23000858503614
3
0.5241950506809246
(24, 24)
0.4235107904007305
(24, 24)
-20.57347928960226
-20.573479387614945
-10.43856997680566
4
0.30228867890819155
(24, 24)
0.3874046269247451
(24, 24)
-20.573479253099944
-20.573479152804424
-10.427052956033236
5
0.49223611774200027
(24, 24)
0.404817779972707
(24, 24)
-12.86761272226589
-20.573479379234627
-9.93956895414361
6
0.30093805497496223
(24, 24)
0.4084699432290132
(24, 24)
-20.57347925908588
-20.573479155527764
-9.0401882958207
7
0.5782015102081953
(24, 24)
0.49498555488432167
(24, 24)
-13.45813498174459
-20.573479441412342
-11.131529506090464
8
0.4491730854594228
(24, 24)
0.49845821775436744
(24, 24)
-20.573479351471224
-20.573479355378556
-9.906812373716631
9
0.9017743187244731
(24, 24)
0.5636515011728322
(24, 24)
-20.5

In [30]:
for cmap_type in cmap_types:
    print(np.array(cmap_type.grid._data).min())

-10.45764016810443
-10.23000858503614
-10.43856997680566
-10.427052956033236
-9.93956895414361
-9.0401882958207
-11.131529506090464
-9.906812373716631
-10.634902691105175
-11.344096118480381
-11.390737392295602
-11.247477496445468
-11.309617059494844
-11.143386181838286
-10.328712776360447
-10.973411055420662
-10.936135866964387
-9.891419998557067
-11.342185641282338
-11.354053129787008
-8.654739533310032
-10.655182054673354
-11.260026282158437
-9.576451127587285
-11.134340610062358
-9.609318160473377
-10.626434498338844
-10.933691690396996
-9.59756180365623
-10.799146733553986
-9.145846192953654
-8.946229703294462
-10.839462928818655
-10.108973614145908
-10.69406603148364
-9.414141929113558
-10.911262639872987
-8.843757426322307
-8.627858581069244
-3.937628956248586
-10.074293870955058
-10.993085955125391
-10.054606407818017
-10.57107941498729
-10.351966057642516
-8.078374747782888
-10.252824279276087
-10.888184811086267
-10.229831979811742
-10.386505720617036
-5.446443939748666
-6.61

In [31]:
pmd_top.cmap_types.clear()
pmd_top.cmaps.clear()

pmd_top.cmap_types.extend(cmap_types)
pmd_top.cmap_types.claim()
pmd_top.cmap_types.index_members()

pmd_top.cmaps.extend(cmap_objs)
pmd_top.cmaps.claim()
pmd_top.cmaps.index_members()

In [32]:
pmd_top.save("cmap_topol.top", overwrite=True)

# TODO
- we need to add the posre.itp include manually

In [35]:
# pmd_top.write("cmap_topol.top")
